In [552]:
import pandas as pd
import math 
import numpy as np
import sys
import os

sys.path.append('..')
import utils.file_utils as ut

ground_truth_path = os.path.join( '..', 'data','table-matching-ground-truth', 'ground-truth')

from gdc.gdc_api import GDCSchema
from table_formatting.column_representation import ColumnRepresentation, gdc_target_columns, gdc_df



In [553]:
dft = pd.read_csv('../data/target.csv')
print(dft.shape)
dft.head(5)

(1068, 17)


,case_submitter_id,age_at_diagnosis,race,ethnicity,gender,vital_status,ajcc_pathologic_t,ajcc_pathologic_n,ajcc_pathologic_stage,tumor_grade,tumor_focality,tumor_largest_dimension_diameter,primary_diagnosis,morphology,tissue_or_organ_of_origin,tumor_code,study
0,01BR001,20089.0,black or african american,not hispanic or latino,female,Alive,T2,N1c,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA,Krug
1,01BR008,17532.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA,Krug
2,01BR009,23376.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA,Krug
3,01BR010,23741.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA,Krug
4,01BR015,12784.0,white,not hispanic or latino,female,Alive,T2,N1,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA,Krug


In [554]:
dfliu = pd.read_csv('../data/liuData_Supplementary_Table_1.csv')
print(dfliu.shape)
dfliu.head(5)

(1185, 83)


,Case_ID,tumor_code,tumor_sample_id_protein,normal_sample_id_protein,is_excluded_from_pancancer_studies,reason_for_exclusion,is_gtex,specimen/aliquout_id_protein_normal,specimen/aliquout_id_protein_tumor,specimen/aliquout_id_RNA_Tumor,...,follow-up/additional_treatment_radiation_therapy_for_new_tumor,follow-up/additional_treatment_pharmaceutical_therapy_for_new_tumor,follow-up/additional_treatment_immuno_for_new_tumor,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor_event_loco-regional,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor_event_metastasis,"Recurrence-free survival, days","Recurrence status (1, yes; 0, no)","Overall survival, days","Survival status (1, dead; 0, alive)","Time between collection and diagnosis, days"
0,11BR047,BRCA,11BR047-T,NaN,NaN,NaN,0.0,NaN,81116212-b7e6-454b-9579-105cf3,ef52c640-13a9-4855-9ce2-0be77a_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,369.0,0.0,NaN
1,11BR043,BRCA,11BR043-T,NaN,NaN,NaN,0.0,NaN,6d34d499-167e-42aa-9790-316fca_D2,6d34d499-167e-42aa-9790-316fca_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,374.0,0.0,NaN
2,11BR049,BRCA,11BR049-T,NaN,NaN,NaN,0.0,NaN,2e700669-85b0-43fa-a9c7-3eaf5a_D2,2e700669-85b0-43fa-a9c7-3eaf5a_D1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,379.0,0.0,NaN
3,11BR023,BRCA,11BR023-T,NaN,NaN,NaN,0.0,NaN,0a80d3c4-0758-447a-958c-ea868c,079b5600-6afc-4785-bb22-48cfab_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,398.0,0.0,NaN
4,18BR010,BRCA,18BR010-T,NaN,NaN,NaN,0.0,NaN,0bb9d596-774e-452b-9c89-a6643c_D2,0bb9d596-774e-452b-9c89-a6643c_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,410.0,0.0,NaN


In [555]:
dfkrug = pd.read_csv('../data/extracted-tables/Krug_A)_Metadata.csv')

# dfkrug = dfkrug.select_dtypes(exclude=['float64', 'int64'])
# threshold = len(dfkrug) * 0.80
# dfkrug = dfkrug.dropna(thresh=threshold, axis=1)

print(dfkrug.shape)
dfkrug.head(5)

(122, 54)


,Sample.ID,Sample.IDs,TMT.Plex,TMT.Channel,Tumor.Stage,Ischemia.Time.in.Minutes,PAM50,NMF.Cluster,NMF.Cluster.Membership.Score,Age.in.Month,...,PIK3CA.Mutation.Status,PTEN.Mutation.Status,MAP3K1.Mutation.Status,AKT1.Mutation.Status,GATA3.Mutation.Status,CBFB.Mutation.Status,KMT2C.Mutation.Status,SF3B1.Mutation.Status,ARID1A.Mutation.Status,MLLT4.Mutation.Status
0,CPT000814,CPT000814,13,127C,Stage IIA,NaN,Basal,Basal-I,1.000,unknown,...,0,0,0,0,0,0,0,0,0,0
1,CPT001846,CPT001846,12,128C,Stage III,NaN,Basal,Basal-I,0.672,unknown,...,1,0,0,0,0,0,0,0,0,0
2,X01BR001,X01BR001,2,129N,Stage IIB,0.0,Basal,Basal-I,0.782,660,...,0,0,0,0,0,0,0,0,0,0
3,X01BR008,X01BR008,16,127C,unknown,NaN,Basal,Basal-I,0.958,unknown,...,0,0,0,0,0,0,0,0,0,0
4,X01BR009,X01BR009,16,127N,unknown,NaN,Basal,Basal-I,0.825,unknown,...,0,0,0,0,0,0,0,0,0,0


In [556]:
dfkrug['Sample.ID'] = dfkrug['Sample.ID'].str.replace('^X', '', regex=True)
dfkrug.head(5)


,Sample.ID,Sample.IDs,TMT.Plex,TMT.Channel,Tumor.Stage,Ischemia.Time.in.Minutes,PAM50,NMF.Cluster,NMF.Cluster.Membership.Score,Age.in.Month,...,PIK3CA.Mutation.Status,PTEN.Mutation.Status,MAP3K1.Mutation.Status,AKT1.Mutation.Status,GATA3.Mutation.Status,CBFB.Mutation.Status,KMT2C.Mutation.Status,SF3B1.Mutation.Status,ARID1A.Mutation.Status,MLLT4.Mutation.Status
0,CPT000814,CPT000814,13,127C,Stage IIA,NaN,Basal,Basal-I,1.000,unknown,...,0,0,0,0,0,0,0,0,0,0
1,CPT001846,CPT001846,12,128C,Stage III,NaN,Basal,Basal-I,0.672,unknown,...,1,0,0,0,0,0,0,0,0,0
2,01BR001,X01BR001,2,129N,Stage IIB,0.0,Basal,Basal-I,0.782,660,...,0,0,0,0,0,0,0,0,0,0
3,01BR008,X01BR008,16,127C,unknown,NaN,Basal,Basal-I,0.958,unknown,...,0,0,0,0,0,0,0,0,0,0
4,01BR009,X01BR009,16,127N,unknown,NaN,Basal,Basal-I,0.825,unknown,...,0,0,0,0,0,0,0,0,0,0


In [557]:
merged_df = dft.merge(dfliu, left_on='case_submitter_id', right_on='Case_ID')
merged_df = merged_df.merge(dfkrug, left_on='case_submitter_id', right_on='Sample.ID')
case_submittter_ids = merged_df['case_submitter_id'].to_list()

In [558]:
id = case_submittter_ids[0]

liu_rec = dfliu[dfliu['Case_ID'] == id]
krug_rec = dfkrug[dfkrug['Sample.ID'] == id]
target_rec = dft[dft['case_submitter_id'] == id]


In [559]:
liu_rec

,Case_ID,tumor_code,tumor_sample_id_protein,normal_sample_id_protein,is_excluded_from_pancancer_studies,reason_for_exclusion,is_gtex,specimen/aliquout_id_protein_normal,specimen/aliquout_id_protein_tumor,specimen/aliquout_id_RNA_Tumor,...,follow-up/additional_treatment_radiation_therapy_for_new_tumor,follow-up/additional_treatment_pharmaceutical_therapy_for_new_tumor,follow-up/additional_treatment_immuno_for_new_tumor,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor_event_loco-regional,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor_event_metastasis,"Recurrence-free survival, days","Recurrence status (1, yes; 0, no)","Overall survival, days","Survival status (1, dead; 0, alive)","Time between collection and diagnosis, days"
14,01BR001,BRCA,01BR001-T,NaN,NaN,NaN,0.0,NaN,17d55745-4ab3-4a80-a182-af938b_D2,17d55745-4ab3-4a80-a182-af938b_D7,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,421.0,0.0,NaN


In [560]:
krug_rec

,Sample.ID,Sample.IDs,TMT.Plex,TMT.Channel,Tumor.Stage,Ischemia.Time.in.Minutes,PAM50,NMF.Cluster,NMF.Cluster.Membership.Score,Age.in.Month,...,PIK3CA.Mutation.Status,PTEN.Mutation.Status,MAP3K1.Mutation.Status,AKT1.Mutation.Status,GATA3.Mutation.Status,CBFB.Mutation.Status,KMT2C.Mutation.Status,SF3B1.Mutation.Status,ARID1A.Mutation.Status,MLLT4.Mutation.Status
2,01BR001,X01BR001,2,129N,Stage IIB,0.0,Basal,Basal-I,0.782,660,...,0,0,0,0,0,0,0,0,0,0


In [561]:
target_rec

,case_submitter_id,age_at_diagnosis,race,ethnicity,gender,vital_status,ajcc_pathologic_t,ajcc_pathologic_n,ajcc_pathologic_stage,tumor_grade,tumor_focality,tumor_largest_dimension_diameter,primary_diagnosis,morphology,tissue_or_organ_of_origin,tumor_code,study
0,01BR001,20089.0,black or african american,not hispanic or latino,female,Alive,T2,N1c,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA,Krug


In [562]:
liu_rec_columns = set(dfliu.columns.str.lower())
krug_rec_columns = set(dfkrug.columns.str.lower())
target_rec_columns = set(dft.columns.str.lower())

common_columns = liu_rec_columns.intersection(target_rec_columns)
print('Liu and GDCTarget common columns', common_columns)

common_columns = liu_rec_columns.intersection(krug_rec_columns)
print('Liu and Krug common columns', common_columns)

common_columns = krug_rec_columns.intersection(target_rec_columns)
print('Krug and GDCTarget common columns', common_columns)


Liu and GDCTarget common columns {'race', 'tumor_code', 'ethnicity'}
Liu and Krug common columns {'ethnicity'}
Krug and GDCTarget common columns {'gender', 'ethnicity'}


In [563]:
# null_representations = ['unknown', 'not performed', 'nan', 'Not Reported'] 
null_representations =  []
def print_non_null_values(df):
    for column in df.columns[:20]:
        value =  df[column].iloc[0]     
        if str(value) not in null_representations:
            print(column,':', value)
print('Krug Values')
print_non_null_values(krug_rec)
print('\nTarget Values')
print_non_null_values(target_rec)
# print('\nLiu Values')
# print_non_null_values(liu_rec)

Krug Values
Sample.ID : 01BR001
Sample.IDs : X01BR001
TMT.Plex : 2
TMT.Channel : 129N
Tumor.Stage : Stage IIB
Ischemia.Time.in.Minutes : 0.0
PAM50 : Basal
NMF.Cluster : Basal-I
NMF.Cluster.Membership.Score : 0.782
Age.in.Month : 660
Gender : female
Ethnicity : black.or.african.american
ER.Updated.Clinical.Status : negative
PR.Clinical.Status : negative
ERBB2.Updated.Clinical.Status : negative
TNBC.Updated.Clinical.Status : positive
ERBB2.Proteogenomic.Status : negative
TOP2A.Proteogenomic.Status : negative
ERBB2.Gene.Amplified : 0
TOP2A.Gene.Amplified : 0

Target Values
case_submitter_id : 01BR001
age_at_diagnosis : 20089.0
race : black or african american
ethnicity : not hispanic or latino
gender : female
vital_status : Alive
ajcc_pathologic_t : T2
ajcc_pathologic_n : N1c
ajcc_pathologic_stage : Stage II
tumor_grade : GX
tumor_focality : Not Reported
tumor_largest_dimension_diameter : Not Reported
primary_diagnosis : Invasive carcinoma of no special type
morphology : 8500/3
tissue_or_

In [564]:
groundtruth_path = os.path.join(ground_truth_path, 'Krug.csv')
groundtruth = ut.load_table_matching_groundtruth(groundtruth_path)
groundtruth

[('Tumor.Stage', 'ajcc_pathologic_stage'),
 ('Age.in.Month', 'age_at_diagnosis'),
 ('Gender', 'gender'),
 ('Ethnicity', 'ethnicity')]

In [565]:
gdc_df
gdc_representation = gdc_df.ajcc_pathologic_stage.unique()
print('GDC Domain:', gdc_representation)
gdc_representation = list(gdc_representation)

GDC Domain: ['stage ib2' 'stage iii' 'stage ia2' 'stage iia1' 'stage iv' 'stage iib'
 'stage ib1' 'stage iiib' 'stage iiid' 'stage iiic2' 'stage iia2'
 'stage iiia2' 'stage ivc' 'stage iiia1' 'stage tis' 'stage 0' 'stage ia1'
 'stage iia' 'stage is' 'stage x' 'stage i' 'stage ivb' 'stage ib'
 'stage ic' 'stage ia' 'stage iiic' 'stage 0is' 'stage ii' 'stage 0a'
 'stage iiic1' 'stage iic' 'stage ia3' 'stage iva' 'stage iiia' <NA>]


In [566]:
# tumor_stage = krug_rec.loc[0, "Tumor.Stage"]
# print(tumor_stage)
# print(tumor_stage.lower() in gdc_representation)
krug_rec["Tumor.Stage"]

2    Stage IIB
Name: Tumor.Stage, dtype: object

In [567]:
# ajcc_pathologic_stage = target_rec.loc[0, "ajcc_pathologic_stage"]
# print(ajcc_pathologic_stage)
# print(ajcc_pathologic_stage.lower() in gdc_representation)
target_rec["ajcc_pathologic_stage"]

0    Stage II
Name: ajcc_pathologic_stage, dtype: object

In [569]:
liu_rec['baseline/tumor_stage_pathological']

14    Stage II
Name: baseline/tumor_stage_pathological, dtype: object